<a href="https://colab.research.google.com/github/ahmarhafeez1/GRADIO-APP-Story-with-Narration-and-Theme-Music/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GRADIO APP: Story with Narration and Theme Music:

Making sure we are utilzing the GPU

In [2]:
!nvidia-smi

Fri Sep  1 06:38:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install --upgrade --quiet pip
!pip install --quiet git+https://github.com/huggingface/transformers.git datasets[audio]
!pip install gTTS
!pip install gradio
!pip install pydub
!pip install nltk
!pip install openai
!pip install torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.1 MB/s eta 0:00:00
  Obtaining dependency information for gradio from https://files.pythonhosted.org/packages/df/e7/e0b548208ff5db6323ad974f094e9435adb0a377f35274196fb74adaf58a/gradio-3.41.2-py3-none-any.whl.metadata
  Obtaining dependency info

In [4]:
from datasets import load_dataset
from transformers import AutoProcessor
from transformers import MusicgenForConditionalGeneration
import torch
from pydub import AudioSegment
import gradio as gr
import openai
import requests
import numpy as np
import scipy

In [5]:
OPENAI_API_KEY = ""
XILABS_API_KEY = ""

In [6]:
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

In [7]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device);

In [8]:
audio_length_in_s = 256 / model.config.audio_encoder.frame_rate
audio_length_in_s

5.12

In [9]:
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")

In [10]:
dataset = load_dataset("sanchit-gandhi/gtzan", split="train", streaming=True)
sample = next(iter(dataset))["audio"]
sampling_rate = model.config.audio_encoder.sampling_rate
# take the first half of the audio sample
sample["array"] = sample["array"][: len(sample["array"]) // 2]




In [11]:
openai.api_key = OPENAI_API_KEY



In [12]:

def get_story(prompt):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": f"You are a professional story teller and you will have to write a detailed story. Please Generate a Story about the following {prompt}"},
            ]
        )
        response_message = response["choices"][0]["message"]
        if response_message["role"] == "assistant":
            return response_message["content"]

    except Exception as e:
        return str(e)


In [13]:
def get_music_description(story):
   try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": f"You are a Audio and you will have to give text descirption for the theme song of a story. Please Generate a Generate One Line Audio Description about the following Story: {story}"},
            ]
        )
        response_message = response["choices"][0]["message"]
        if response_message["role"] == "assistant":
            return response_message["content"]

   except Exception as e:
        return str(e)


In [14]:
sampling_rate = model.config.audio_encoder.sampling_rate

In [15]:
def get_bgm(prompt):
  file = "audio.wav"
  from transformers import AutoProcessor
  processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
  inputs = processor(
      text=[prompt,],
      padding=True,
      return_tensors="pt",
  )
  audio_values = model.generate(**inputs.to(device), do_sample=True, guidance_scale=3, max_new_tokens=256)
  #scipy.io.wavfile.write(file, rate=sampling_rate, data=,)
  return sampling_rate,audio_values[0,0].cpu().numpy()

In [16]:
def get_narration(story):
  file = "narration.mp3"
  CHUNK_SIZE = 1024
  url = "https://api.elevenlabs.io/v1/text-to-speech/XB0fDUnXU5powFXDhCwa"
  headers = {
  "Accept": "audio/mpeg",
  "Content-Type": "application/json",
  "xi-api-key": XILABS_API_KEY
  }

  data = {
    "text": story,
    "model_id": "eleven_monolingual_v1",
    "voice_settings": {
      "stability": 0.5,
      "similarity_boost": 0.5
    }
  }

  response = requests.post(url, json=data, headers=headers)
  with open(file, 'wb') as f:
      for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
          if chunk:
              f.write(chunk)
  return file


In [17]:
def generate_story_bgs(prompt):
  story = get_story(prompt)
  music_des = get_music_description(story)
  bgm = get_bgm(music_des)
  narration = get_narration(story)
  return story , bgm, narration

In [18]:
iface = gr.Interface(
    fn=generate_story_bgs,
    inputs=[gr.inputs.Textbox(type='text', label="What do you want your story to be about?")],
    outputs=[
        gr.outputs.Textbox(type='text', label="Story will appear here"),
        gr.outputs.Audio(type="numpy",label="Theme Music Will Appear here"),
        gr.outputs.Audio(type="filepath",label="Narration")
        ],
    live=False
)

iface.queue().launch(share=True, debug=True)

<ipython-input-18-7304737f016b>:3: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=[gr.inputs.Textbox(type='text', label="What do you want your story to be about?")],
<ipython-input-18-7304737f016b>:3: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=[gr.inputs.Textbox(type='text', label="What do you want your story to be about?")],
<ipython-input-18-7304737f016b>:3: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  inputs=[gr.inputs.Textbox(type='text', label="What do you want your story to be about?")],
<ipython-input-18-7304737f016b>:5: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  gr.outputs.Textbox(type='text', label="Story will appear here"),
<ipython-input-18-7304737f016b>:6

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://b2cd54881a2ee10efd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/gradio/processing_utils.py:183: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b2cd54881a2ee10efd.gradio.live
